# Gold Layer Analytics
## Business Metrics and Insights

This notebook creates the Gold layer tables and analytics:
1. Vehicle maintenance costs and trends
2. Service type analysis
3. Maintenance schedule compliance
4. Cost optimization insights
5. Predictive maintenance indicators

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import plotly.express as px
import pandas as pd

# Initialize Spark session
spark = SparkSession.builder \
    .appName("VehicleMaintenance-Gold") \
    .getOrCreate()

# Set database
spark.sql("USE vehicle_maintenance")

In [ ]:
# Create gold tables for analytics
# 1. Monthly maintenance costs
spark.sql("""
CREATE TABLE IF NOT EXISTS vehicle_maintenance.gold_monthly_costs (
    year_month DATE,
    total_cost DOUBLE,
    total_labor_cost DOUBLE,
    total_parts_cost DOUBLE,
    avg_cost_per_service DOUBLE,
    number_of_services BIGINT,
    processing_timestamp TIMESTAMP
)
USING DELTA
LOCATION '/mnt/vehicle-data/delta/gold/monthly_costs'
""")

# 2. Vehicle maintenance metrics
spark.sql("""
CREATE TABLE IF NOT EXISTS vehicle_maintenance.gold_vehicle_metrics (
    vehicle_id STRING,
    last_maintenance_date DATE,
    next_maintenance_due DATE,
    maintenance_status STRING,
    total_maintenance_cost DOUBLE,
    avg_maintenance_interval_days DOUBLE,
    number_of_services BIGINT,
    processing_timestamp TIMESTAMP
)
USING DELTA
LOCATION '/mnt/vehicle-data/delta/gold/vehicle_metrics'
""")

# 3. Service type analysis
spark.sql("""
CREATE TABLE IF NOT EXISTS vehicle_maintenance.gold_service_analysis (
    service_category STRING,
    service_type STRING,
    total_services BIGINT,
    avg_cost DOUBLE,
    avg_mileage_at_service DOUBLE,
    most_common_parts ARRAY<STRING>,
    processing_timestamp TIMESTAMP
)
USING DELTA
LOCATION '/mnt/vehicle-data/delta/gold/service_analysis'
""")

In [ ]:
# Function to generate monthly cost analytics
def calculate_monthly_costs():
    monthly_costs = spark.sql("""
        SELECT
            date_trunc('month', maintenance_date) as year_month,
            sum(cost) as total_cost,
            sum(labor_cost) as total_labor_cost,
            sum(parts_cost) as total_parts_cost,
            avg(cost) as avg_cost_per_service,
            count(*) as number_of_services,
            current_timestamp() as processing_timestamp
        FROM vehicle_maintenance.silver_maintenance
        GROUP BY date_trunc('month', maintenance_date)
    """)
    
    monthly_costs.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("vehicle_maintenance.gold_monthly_costs")
    
    return monthly_costs

# Function to calculate vehicle-specific metrics
def calculate_vehicle_metrics():
    vehicle_metrics = spark.sql("""
        SELECT
            vehicle_id,
            max(maintenance_date) as last_maintenance_date,
            max(next_maintenance_date) as next_maintenance_due,
            CASE
                WHEN max(next_maintenance_date) < current_date() THEN 'Overdue'
                WHEN max(next_maintenance_date) < date_add(current_date(), 30) THEN 'Due Soon'
                ELSE 'On Schedule'
            END as maintenance_status,
            sum(cost) as total_maintenance_cost,
            avg(datediff(
                maintenance_date,
                lag(maintenance_date) over (partition by vehicle_id order by maintenance_date)
            )) as avg_maintenance_interval_days,
            count(*) as number_of_services,
            current_timestamp() as processing_timestamp
        FROM vehicle_maintenance.silver_maintenance
        GROUP BY vehicle_id
    """)
    
    vehicle_metrics.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("vehicle_maintenance.gold_vehicle_metrics")
    
    return vehicle_metrics

# Function to analyze service patterns
def analyze_services():
    service_analysis = spark.sql("""
        SELECT
            service_category,
            service_type,
            count(*) as total_services,
            avg(cost) as avg_cost,
            avg(mileage) as avg_mileage_at_service,
            collect_set(explode(parts_used)) as most_common_parts,
            current_timestamp() as processing_timestamp
        FROM vehicle_maintenance.silver_maintenance
        GROUP BY service_category, service_type
    """)
    
    service_analysis.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("vehicle_maintenance.gold_service_analysis")
    
    return service_analysis

# Generate all analytics
try:
    monthly_df = calculate_monthly_costs()
    vehicle_df = calculate_vehicle_metrics()
    service_df = analyze_services()
    print("Gold layer analytics generated successfully")

## Business Insights Visualization

Create visualizations for key maintenance metrics:
1. Monthly cost trends
2. Service type distribution
3. Maintenance status by vehicle
4. Cost optimization opportunities

In [ ]:
# Convert monthly costs to pandas for visualization
monthly_pd = monthly_df.toPandas()

# Create monthly cost trend plot
fig1 = px.line(monthly_pd, 
               x='year_month', 
               y=['total_cost', 'total_labor_cost', 'total_parts_cost'],
               title='Monthly Maintenance Costs')
fig1.show()

# Create service category distribution
service_pd = service_df.toPandas()
fig2 = px.pie(service_pd, 
              values='total_services', 
              names='service_category',
              title='Service Type Distribution')
fig2.show()

# Vehicle maintenance status
vehicle_pd = vehicle_df.toPandas()
fig3 = px.bar(vehicle_pd, 
              x='maintenance_status', 
              y='number_of_services',
              title='Maintenance Status Distribution')
fig3.show()

# Cost per service type
fig4 = px.bar(service_pd,
              x='service_type',
              y='avg_cost',
              color='service_category',
              title='Average Cost by Service Type')
fig4.show()